Nama      : Muhammad Fawwaz Naabigh\
Nim       : 13519206\
Asal Univ : ITB\
Tanggal   : 14 November 2020\
Sumber    : [Titanic: Machine Learning from Disaster
](https://www.kaggle.com/c/titanic/data)

##### Overview
Dataset berisi data penumpang kapal Titanic yang tenggelam dengan nilai 1 pada kolom 'survived' jika penumpang selamat dan 0 jika tidak.

## Data Observation

In [307]:
# import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [308]:
# load dataset
train = pd.read_csv("train_titanic.csv")
test = pd.read_csv("test_titanic.csv")

In [309]:
# basic statistical details dari data latih
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [310]:
# basic statistical details dari data uji
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [311]:
# cek data kosong
print('Data Latih')
print('-'*20)
print(train.isnull().sum())
print('-'*30)
print('Data Uji')
print('-'*20)
print(test.isnull().sum())

Data Latih
--------------------
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
------------------------------
Data Uji
--------------------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


## Data Preprocessing

In [312]:
# analisis berdasarkan jenis kelamin
train[['Sex', 'Survived']].groupby('Sex').mean().sort_values(by='Survived', ascending=False)

,Survived
Sex,
female,0.742038
male,0.188908


In [313]:
# analisis berdasarkan jumlah saudara/pasangan 
train[['SibSp', 'Survived']].groupby('SibSp').mean().sort_values(by='Survived', ascending=False)

,Survived
SibSp,
1,0.535885
2,0.464286
0,0.345395
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [314]:
# analisis berdasarkan jumlah orangtua/anak
train[['Parch', 'Survived']].groupby('Parch').mean().sort_values(by='Survived', ascending=False)

,Survived
Parch,
3,0.600000
1,0.550847
2,0.500000
0,0.343658
5,0.200000
4,0.000000
6,0.000000


In [315]:
# analisis berdasarkan lokasi keberangkatan (C = Cherbourg, Q = Queenstown, S = Southampton)
train[['Embarked', 'Survived']].groupby('Embarked').mean().sort_values(by='Survived', ascending=False)

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [316]:
# mengisi value usia yang kosong dengan '-0.5'
train['Age'] = train['Age'].fillna(-.5)
test['Age'] = test['Age'].fillna(-.5)

In [317]:
# pengkategorian usia berdasarkan bins
bins = [0, 1, 5, 12, 18, 25, 30, 60, np.inf]
labels = ['unknown', 'balita', 'anak', 'remaja', 'pemuda', 'dewasa', 'paruh baya', 'tua']

train['AgeGroup'] = pd.cut(train['Age'], bins=bins, labels=labels)
test['AgeGroup'] = pd.cut(test['Age'], bins=bins, labels=labels)

In [318]:
# analisis berdasarkan kategori usia
train[['AgeGroup', 'Survived']].groupby('AgeGroup').mean().sort_values(by='Survived', ascending=False)

,Survived
AgeGroup,
unknown,0.857143
balita,0.633333
remaja,0.428571
paruh baya,0.420495
dewasa,0.388889
anak,0.360000
pemuda,0.333333
tua,0.227273


In [319]:
# hapus kolom yang tidak diperlukan
train = train.drop(['Cabin', 'Ticket'], axis=1)
test = test.drop(['Cabin', 'Ticket'],axis=1)

In [320]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,pemuda
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,paruh baya
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,dewasa
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,paruh baya
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,paruh baya


In [321]:
# ekstrak gelar/title penumpang
combine = [train, test]

for data in combine:
    data['Title'] = data.Name.str.extract(' ([a-zA-z]+)\.', expand=False)

In [322]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,pemuda,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,paruh baya,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,dewasa,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,paruh baya,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,paruh baya,Mr


In [323]:
train['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer'], dtype=object)

In [324]:
# penggolongan gelar
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Capt', 'Col',
    'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    
    dataset['Title'] = dataset['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [325]:
# drop nama penumpang
train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)

In [326]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup,Title
0,1,0,3,male,22.0,1,0,7.2500,S,pemuda,Mr
1,2,1,1,female,38.0,1,0,71.2833,C,paruh baya,Mrs
2,3,1,3,female,26.0,0,0,7.9250,S,dewasa,Miss
3,4,1,1,female,35.0,1,0,53.1000,S,paruh baya,Mrs
4,5,0,3,male,35.0,0,0,8.0500,S,paruh baya,Mr


In [327]:
train['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rare', 'Royal'], dtype=object)

In [328]:
# analisis berdasarkan gelar
train[['Title', 'Survived']].groupby('Title').mean().sort_values(by='Survived', ascending=False)

,Survived
Title,
Royal,1.000000
Mrs,0.793651
Miss,0.702703
Master,0.575000
Rare,0.285714
Mr,0.156673


In [329]:
# mapping gelar menjadi data numeric, dengan angka terkecil mewakili survival rate tertinggi
title_mapping = {"Royal": 1, "Mrs": 2, "Miss": 3, "Master": 4, "Rare": 5, "Mr": 6}
train['Title'] = train['Title'].replace(title_mapping)
test['Title'] = test['Title'].replace(title_mapping)

In [330]:
# cek data kosong
print('Data Latih')
print('-'*20)
print(train.isnull().sum())
print('-'*30)
print('Data Uji')
print('-'*20)
print(test.isnull().sum())

Data Latih
--------------------
PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         2
AgeGroup       177
Title            0
dtype: int64
------------------------------
Data Uji
--------------------
PassengerId     0
Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            1
Embarked        0
AgeGroup       86
Title           0
dtype: int64


In [331]:
# cek kategori usia terbanyak per title
train[train['Title']==1]['AgeGroup'].mode()

0    paruh baya
Name: AgeGroup, dtype: category
Categories (8, object): [unknown < balita < anak < remaja < pemuda < dewasa < paruh baya < tua]

In [332]:
train[train['Title']==2]['AgeGroup'].mode()

0    paruh baya
Name: AgeGroup, dtype: category
Categories (8, object): [unknown < balita < anak < remaja < pemuda < dewasa < paruh baya < tua]

In [333]:
train[train['Title']==3]['AgeGroup'].mode()

0    pemuda
Name: AgeGroup, dtype: category
Categories (8, object): [unknown < balita < anak < remaja < pemuda < dewasa < paruh baya < tua]

In [334]:
train[train['Title']==4]['AgeGroup'].mode()

0    balita
1      anak
Name: AgeGroup, dtype: category
Categories (8, object): [unknown < balita < anak < remaja < pemuda < dewasa < paruh baya < tua]

In [335]:
train[train['Title']==5]['AgeGroup'].mode()

0    paruh baya
Name: AgeGroup, dtype: category
Categories (8, object): [unknown < balita < anak < remaja < pemuda < dewasa < paruh baya < tua]

In [336]:
train[train['Title']==6]['AgeGroup'].mode()

0    paruh baya
Name: AgeGroup, dtype: category
Categories (8, object): [unknown < balita < anak < remaja < pemuda < dewasa < paruh baya < tua]

In [337]:
# pengkategorian title ke usia berdasarkan kemunculan kategori usia terbanyak per title 
age_map = {1: 'paruh baya', 2:'paruh baya', 3:'pemuda', 4:'balita', 5:'paruh baya', 6:'paruh baya'}

In [339]:
# mengosongkan kategori usia 'unknown'
train["AgeGroup"] = train["AgeGroup"].replace({'unknown':np.nan})
test["AgeGroup"] = test["AgeGroup"].replace({'unknown':np.nan})

In [340]:
# mengisi kategori usia yang kosong dengan memperkirakan sesuai jabatan
train = train.fillna({"AgeGroup": train["Title"].map(age_map)})
test = test.fillna({"AgeGroup": test["Title"].map(age_map)})

In [343]:
# drop kolom usia
train = train.drop('Age', axis=1)
test = test.drop('Age', axis=1)

In [344]:
# analisis berdasarkan kategori usia
train[['AgeGroup', 'Survived']].groupby('AgeGroup').mean().sort_values(by="Survived", ascending=False)

,Survived
AgeGroup,
balita,0.659091
remaja,0.428571
pemuda,0.396040
dewasa,0.388889
anak,0.360000
paruh baya,0.350000
tua,0.227273


In [345]:
# mengubah kategori usia menjadi data numeric
agegroup_map ={'balita':1, 'remaja':2, 'pemuda':3, 'dewasa':4, 'anak':5, 'paruh baya':6, 'tua':7}
train["AgeGroup"] = train["AgeGroup"].replace(agegroup_map)
test["AgeGroup"] = test["AgeGroup"].replace(agegroup_map)

In [346]:
train[['AgeGroup', 'Survived']].groupby('AgeGroup').mean().sort_values(by="Survived", ascending=False)

,Survived
AgeGroup,
1,0.659091
2,0.428571
3,0.396040
4,0.388889
5,0.360000
6,0.350000
7,0.227273


In [347]:
# analisis embarked
train[['Embarked', 'Survived']].groupby('Embarked').mean().sort_values(by="Survived", ascending=False)

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [348]:
# mengubah kolom embarked menjadi numeric
embarked_map = {'C':1, 'Q':2, 'S':3}
train = train.fillna({'Embarked':'S'})
train['Embarked'] = train['Embarked'].replace(embarked_map)
test['Embarked'] = test['Embarked'].replace(embarked_map)

In [352]:
# mengisi kolom fare yang kosong berdasarkan rata-rata Pclass
for i in range(len(test['Fare'])):
    if pd.isnull(test['Fare'].iloc[i]):
        pclass = test['Pclass'].iloc[i]
        test['Fare'].iloc[i] = round(test[test['Pclass'] == pclass]['Fare'].mean(), 4)

C:\Users\fwznbg\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [353]:
# mengubah kolom sex ke numeric
sex_map = {'male':0, 'female':1}
train['Sex'] = train['Sex'].replace(sex_map)
test['Sex'] = test['Sex'].replace(sex_map)

In [354]:
# mengkategorikan fare berdasarkan quartil
train['FareBand'] = pd.qcut(train['Fare'], 4, labels=[1, 2, 3, 4])
test['FareBand'] = pd.qcut(test['Fare'], 4, labels=[1, 2, 3, 4])

In [355]:
# drop kolom fare
train = train.drop('Fare', axis=1)
test = test.drop('Fare', axis=1)

## Modelling

In [356]:
from sklearn.model_selection import train_test_split

X = train.drop(['Survived', 'PassengerId'], axis=1)
y = train['Survived']

In [357]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [358]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
y_pred = gaussian.predict(X_test)
acc_gaussian = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_gaussian)

80.68


In [359]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

82.03


In [360]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc_svc = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_svc)

84.41


In [361]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_linear_svc)

82.03


C:\Users\fwznbg\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [362]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
acc_perceptron = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_perceptron)

77.63


In [363]:
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(X_train, y_train)
y_pred = decisiontree.predict(X_test)
acc_decisiontree = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_decisiontree)

78.31


In [364]:
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(X_train, y_train)
y_pred = randomforest.predict(X_test)
acc_randomforest = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_randomforest)

80.68


In [365]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_knn)

83.05


In [366]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
acc_sgd = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_sgd)

79.66


In [367]:
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(X_train, y_train)
y_pred = gbk.predict(X_test)
acc_gbk = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_gbk)

82.37


In [371]:
ids = test['PassengerId']
predictions = svc.predict(test.drop('PassengerId', axis=1))

#set the output as a dataframe and convert to csv file named submission.csv
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('prediction.csv', index=False)

In [372]:
models = pd.DataFrame({'model':['gaussian', 'logreg', 'svc', 'linear svc', 'perceptron','DTC','RandForest', 'KNN', 'SGD', 'GradBoost'], 
                       'akurasi':[acc_gaussian, acc_logreg, acc_svc, acc_linear_svc, acc_perceptron, acc_decisiontree, acc_randomforest, acc_knn, acc_sgd, acc_gbk]})

In [373]:
models.sort_values(by='akurasi', ascending=False)

,model,akurasi
2,svc,84.41
7,KNN,83.05
9,GradBoost,82.37
1,logreg,82.03
3,linear svc,82.03
0,gaussian,80.68
6,RandForest,80.68
8,SGD,79.66
5,DTC,78.31
4,perceptron,77.63


## Insight

Golongan yang memiliki survival rate tertinggi
- Perempuan 
- Orang yang pergi bersama 1 orang saudara
- Orang yang pergi bersama 3 orang (orang tua/anak)
- Orang yang berangkat dari Cherbourg
- Bangsawan
- Balita

Dari hasil permodelan metode yang menghasilkan hasil paling maksimum adalah Support Vector Classifier